## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [22]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier                           # GBM
from sklearn.ensemble import RandomForestClassifier# 隨機森林
from sklearn.model_selection import GridSearchCV



# 1. get data
digits = datasets.load_digits()
#digits = datasets.load_breast_cancer()



In [23]:
#2. split data
x_train,x_test,y_train,y_test = train_test_split(digits.data,digits.target,test_size=0.25,random_state=42)

In [30]:
# 3.1 建立Model
#clf = GradientBoostingClassifier ()
clf = GradientBoostingClassifier (loss = 'deviance',learning_rate=0.1, n_estimators=100)

#clf = RandomForestClassifier()
# 3.2 訓練Model
clf.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [31]:
# 4. 預測
y_predict = clf.predict(x_test)

In [32]:
# 5. 評分
metrics.accuracy_score(y_predict,y_test)

0.9711111111111111

In [33]:
# 6.1 
n_estimators = [50, 100, 150]
learning_rate = [0.1, 0.2,0.3]
#max_depth = [1, 3, 5]


param_grid = dict(n_estimators=n_estimators, learning_rate = learning_rate)



# 6.2.1
grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)   #Model
# 6.2.2
grid_result = grid_search.fit(x_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   27.6s finished


In [34]:
grid_result.best_params_

{'learning_rate': 0.1, 'n_estimators': 100}

In [35]:
clf = GradientBoostingClassifier (loss = 'deviance',learning_rate=0.2, n_estimators=50)
#clf = GradientBoostingClassifier(learning_rate=grid_result.best_params_['max_depth'],n_estimators=grid_result.best_params_['n_estimators'])
clf.fit(x_train,y_train)
y_predict = clf.predict(x_test)
metrics.accuracy_score(y_test, y_predict)

0.9733333333333334

Q：分數變低？


因為 GridSearch 是針對特定資料&模型做參數優化，你這邊優化的目標只有 training data。所以用在 testing data反而變差，那 training & testing data 的權衡通常會搭配 cross validation 的方式。官方有一個範例是 GridSearch + Cross-Validation： https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

Q：每次不一樣？
Notes

The features are always randomly permuted at each split. Therefore, the best found split may vary, even with the same training data and max_features=n_features, if the improvement of the criterion is identical for several splits enumerated during the search of the best split. To obtain a deterministic behaviour during fitting, random_state has to be fixed.

意思就是要使用 random_state 才會有固定的結果。另外 Regressor 那邊感謝你的提醒，程式碼已經有修正囉